In [1]:
from pathlib import Path
import re
import sys
import pandas as pd
from typing import List
from functools import cmp_to_key
from collections import Counter
import itertools
import math


In [2]:
# util function
def read_input(x:int)->any:
    """Function to read inputs"""
    data_dir = Path("./data")
    filename = data_dir/f"input_{x}.txt"
    with open(filename,'r') as file:
        data = [x.strip() for x in file.readlines()]
    return data

def pretty_print(q:int, ans_1:int=0, ans_2:int=0)->None:
    print(f"Answer for Ques {q}\nPart 1:\t{ans_1}\nPart 2:\t{ans_2}")

def assert_answer(q:int, ans_1:int=0, ans_2:int=0)->None:
    ans = list(pd.read_csv("results.csv").set_index("question").loc[q])
    assert ans[0] == ans_1, f"Incorrect Q-{q} Part 1: Expected={ans[0]}, Ouput={ans_1}"
    assert ans[1] == ans_2, f"Incorrect Q-{q} Part 2: Expected={ans[1]}, Ouput={ans_2}"


In [3]:
# global variable
numbers_str = ["one", "two", "three", "four", "five", "six", "seven", "eight","nine"]
numbers_map = dict(zip(numbers_str, range(1,10)))
DAY = 1


In [4]:
# decorator
def print_day(day=1):
    def decorator_print(func):
        def wrapper_print(*args, **kwargs):
            global DAY
            DAY = day
            [f_1, f_2] = func(*args, **kwargs)
            ans_1 = f_1()
            ans_2 = f_2()
            pretty_print(day, ans_1, ans_2)
            return [f_1, f_2]
        return wrapper_print
    return decorator_print

def assert_day(day=1):
    def decorator_assert(func):
        def wrapper_assert(*args, **kwargs):
            global DAY
            DAY = day
            [f_1, f_2] = func(*args, **kwargs)
            [ans_1, ans_2] = f_1(), f_2()
            assert_answer(day, ans_1, ans_2)
            return [f_1, f_2]
        return wrapper_assert
    return decorator_assert


In [5]:
# template
"""
# Day N

@assert_day(day=N)
@print_day(day=N)
def solve_day():
    data  = read_input(DAY)
    
    def solve_1() -> int:
        pass

    def solve_2() -> int:
        pass
    
    return [solve_1, solve_2]
_ = solve_day()

""";


In [6]:
# Day 1
@assert_day(day=1)
@print_day(day=1)
def solve_day():
    data  = read_input(DAY)
    def extract_int(elem:str, reg_pattern:str)->int:
        digits = map(lambda x: x.group(1) , re.finditer(reg_pattern, elem) )
        digits = list(map(lambda x: str(numbers_map.get(x,x) ), digits ) )
        if len(digits) == 0:
            return 0
        return int(digits[0] + digits[-1])

    def solve_1() -> int:
        reg_pattern = r"(?=(\d))"
        return sum(map( lambda x: extract_int(x, reg_pattern), data))

    def solve_2() -> int:
        reg_pattern = r"(?=(\d|" + "|".join(numbers_str) + "))"
        return sum(map( lambda x: extract_int(x, reg_pattern), data))
    
    return [solve_1, solve_2]
_ = solve_day()


Answer for Ques 1
Part 1:	54331
Part 2:	54518


In [7]:
# Day 2
@assert_day(day=2)
@print_day(day=2)
def solve_day():
    data  = read_input(DAY)
    def extract_count(row:str)->List[int]:
        f = lambda x: max(map(int, re.findall(f"([0-9]+) {x}", row)))
        return [f("red"), f("green"), f("blue")]

    def is_valid_game(row:str) -> int:
        game = int(re.findall("Game ([0-9]+):", row)[0])
        [r,g,b] = extract_count(row)
        return game if r<=12 and g<=13 and b<=14 else 0

    def power_of_game(row:str)->int:
        [r,g,b] = extract_count(row)
        return r*g*b
    
    def solve_1()->int:
        return sum(map(is_valid_game, data))
    
    def solve_2()->int:
        return sum(map(power_of_game, data))
    
    return [solve_1, solve_2]
_ = solve_day()


Answer for Ques 2
Part 1:	1853
Part 2:	72706


In [8]:
# Day 3
cord_id = 0
@assert_day(day=3)
@print_day(day=3)
def solve_day():
    data = read_input(DAY)
    symbols = [ '\#', '\$', '\%','\&','\*','\+','\-','\/','\=','\@']
    symbols_idx = set()
    numbers_idx = set()
    cord_id_map = dict()
    numbers_map = dict()
   
    for i,elem in enumerate(data):
        symbols_idx.update( set( map(
                    lambda x: (i, x.start()),  
                    re.finditer(r"|".join(symbols), elem))))
        
    def gen_cord(cord:tuple) -> List[tuple]:
        cube = list(itertools.product([-1,0,1], [-1,0,1]))
        return map(lambda x: tuple(map(sum, zip(cord, x))), cube)

    def check_number(cord:tuple, symbols_idx:set) -> int:
        return symbols_idx.intersection(gen_cord(cord))

    def check_asterisk(cord:tuple, number_idx:set) -> int:
        return number_idx.intersection(gen_cord(cord))

    def extract_and_check_numbers(i:int, row:str)->List[int]:
        global cord_id
        ans = []
        for m in re.finditer(r"\d+", row):
            start_cord, end_cord, num = (i,m.start()), (i,m.end()-1), int(m.group(0))

            for x in range(m.start(), m.end()):
                cord = (i,x)
                numbers_idx.update({cord})
                cord_id_map[cord] = cord_id
                numbers_map[cord_id] = num
                numbers_map[cord_id] = num
            cord_id+=1
            if check_number(start_cord, symbols_idx) \
                or check_number(end_cord, symbols_idx):
                ans.append(num)
        return (ans)
    
    def extract_and_check_asterisk(i:int, row:str)->List[int]:
        ans = []
        for m in re.finditer("\*", row):
            start_cord = (i,m.start())
            numbers = check_asterisk(start_cord, numbers_idx)
            cord_ids  = list({cord_id_map[x] for x in numbers})
            if len(cord_ids)==2:
                ans.append( numbers_map[cord_ids[0]] * numbers_map[cord_ids[1]] )
        return (ans)

            
    def solve_1() -> int:
        all_valid_numers = []
        for i, row in enumerate(data):
            all_valid_numers += extract_and_check_numbers(i,row)
        return sum(all_valid_numers)

    def solve_2() -> int:
        all_valid_numers = []
        for i, row in enumerate(data):
            all_valid_numers += extract_and_check_asterisk(i,row)
        return sum(all_valid_numers)
    return [solve_1, solve_2]
_ = solve_day()
del cord_id


Answer for Ques 3
Part 1:	530849
Part 2:	84900879


In [9]:
# Day 4

@assert_day(day=4)
@print_day(day=4)
def solve_day():
    data = read_input(DAY)
    def extract_int(row:str, pattern:str) -> set:
        nums = re.findall(pattern, row)[0].strip()
        return set(map(int, re.split(r"\s+", nums)))

    def winning_points(row:str)->int:
        winning = extract_int(row, "\:([0-9 ]+)\|")
        all_nums = extract_int(row, "\|([0-9 ]+)")
        return len(winning.intersection(all_nums))

    def solve_1() -> int:
        return sum(map(lambda x: int(2**(winning_points(x)-1)), data))

    def solve_2() -> int:
        game_points = list(map(winning_points, data))
        counts = [1]*len(game_points)
        for idx, k in enumerate(game_points):
            for j in range(k):
                if j<len(game_points):
                    counts[idx+j+1] += counts[idx]
        return sum(counts)
        
    return [solve_1, solve_2]
_ = solve_day()


Answer for Ques 4
Part 1:	18619
Part 2:	8063216


In [10]:
# Day 5

@assert_day(day=5)
@print_day(day=5)
def solve_day():
    data = read_input(DAY)
    def generate_maps():
        key = ""
        map_of_map = dict()
        for row in data[1:]:
            if len(row):
                if 'map' in row:
                    key = re.findall("([a-z-]+) map", row)[0]
                if not key in map_of_map:
                    map_of_map[key] = list()
                else:
                    l = list(map(int, row.split(" ")))
                    map_of_map[key].append(list(l))
        return map_of_map

    def find_next(seed, map_of_map, key):
        value = seed
        for l in map_of_map[key]:
            if seed >= l[1] and seed < l[1]+l[2]:
                return seed-l[1]+l[0]
        return value

    def between(a, x,y):
        return a>=x and a < y

    def find_next_list_single(seed, map_of_map, key):
        lst = map_of_map[key]
        res = []
        for (d,s,m) in lst:
            if between(seed[0], s, s+m):
                if between(seed[0]+seed[1]-1, s, s+m):
                        res.append((seed[0]-s+d, seed[1]))
                        return res
                else:
                        res.append( (seed[0]-s+d, m - (seed[0]-s) ) ) 
                        new_start = (s+m, seed[1] - (s+m-seed[0]) )
                        res += find_next_list_single(new_start, map_of_map, key)
                        return res
            elif between(s, seed[0], seed[0]+seed[1]):
                new_list = [(seed[0], s-seed[0]), (s, seed[0]+seed[1]-s)]
                res += find_next_list(new_list, map_of_map, key)
                return res
            else:
                continue
        return [seed]

                    
    def find_next_list(seeds, map_of_map, key):
        next_lst = []
        for seed in seeds:
            next_lst += find_next_list_single(seed, map_of_map, key)
        return next_lst        

    def find_location(func, input):
        map_of_map = generate_maps()
        return func(
            func(
                func(
                    func(
                        func(
                            func( 
                                func(
                                    input,  map_of_map, 
                                    "seed-to-soil"),  map_of_map, 
                                "soil-to-fertilizer"), map_of_map,
                            "fertilizer-to-water"), map_of_map,
                        "water-to-light"), map_of_map,
                    "light-to-temperature"), map_of_map,
                "temperature-to-humidity"), map_of_map,
            "humidity-to-location")

    def get_seeds():
        return list(map(int, re.findall("([\d ]+)", data[0])[0].strip().split(" ")))

    def solve_1():
        seeds = get_seeds()
        return min(map(lambda x: find_location(find_next, x), seeds))

    def solve_2():
        seeds = get_seeds()
        seeds_pair = list(zip(seeds[::2], seeds[1::2]))
        return min(map(lambda x: x[0], find_location(find_next_list, seeds_pair)))
    
    return [solve_1, solve_2]
_ = solve_day()


Answer for Ques 5
Part 1:	282277027
Part 2:	11554135


In [11]:
# Day 6

@assert_day(day=6)
@print_day(day=6)
def solve_day():
    data = read_input(DAY)
    def resolve_ceil(x:int) -> int:
        a = math.ceil(x)
        return a+1 if a==x else a

    def resolve_floor(x:int) -> int:
        a = math.floor(x)
        return a-1 if a==x else a

    def solve_quad(time, distance):
        sqrt_part = time*time-4*distance
        if sqrt_part < 0:
            return 0
        a = resolve_ceil((time-math.sqrt(sqrt_part))/2.0)
        b = resolve_floor((time+math.sqrt(sqrt_part))/2.0)
        return max(0,b-a+1)

    def solver(input):
        return math.prod(
                    map(
                        lambda x: solve_quad(x[0], x[1]), 
                        input
                    )
                )
    def times_distance():
        times = re.findall("(\d+)", data[0])
        distances  = re.findall("(\d+)", data[1])
        return [times, distances]

    def solve_1():
        [times, distances] = times_distance()
        times, distances = map(int, times), map(int, distances)    
        return solver(zip(times, distances))

    def solve_2():
        [time, distance] = times_distance()
        time, distance = int("".join(time)), int("".join(distance))
        return solver([(time, distance)])
        
    return [solve_1, solve_2]
_ = solve_day()


Answer for Ques 6
Part 1:	281600
Part 2:	33875953


In [12]:
# Day N

@assert_day(day=7)
@print_day(day=7)
def solve_day():
    data  = read_input(DAY)

    hand_map = {x.split(' ')[0]:int(x.split(' ')[1]) for x in data}
    cards = ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3',  '2']
    cards_jokers = ['A', 'K', 'Q', 'T', '9', '8', '7', '6', '5', '4', '3',  '2', 'J']
    cards_rank = dict(zip(cards, range(len(cards))))
    cards_rank_joker = dict(zip(cards_jokers, range(len(cards_jokers))))
    with_joker = False

    def resolve_joker(hand):
        counts = Counter(hand)
        sorted_counts = sorted(counts, key=counts.get, reverse=True)
        resovled_char = 'J'
        for i in sorted_counts:
            if i !='J':
                resovled_char = i
                break
        return hand.replace('J', resovled_char)

    def get_hand_rank(hand):
        nonlocal with_joker
        if with_joker:
            hand = resolve_joker(hand)

        sorted_lst = sorted(Counter(hand).values())
        if sorted_lst == [ 5 ]:
            return 1
        elif sorted_lst == [1,4]:
            return 2
        elif sorted_lst == [2,3]:
            return 3
        elif sorted_lst == [1,1,3]:
            return 4
        elif sorted_lst == [1,2,2]:
            return 5
        elif sorted_lst == [1,1,1,2]:
            return 6
        else:
            return 7

    def compare_str_helper(str1, str2, idx):
        nonlocal with_joker
        ranks = cards_rank_joker if with_joker else cards_rank
        if idx >= min(len(str1), len(str2)):
            return 0
        r1, r2 = ranks[str1[idx]], ranks[str2[idx]]
        return r1 - r2 if r1!=r2 else compare_str_helper(str1, str2, idx+1)

    def compare_str( str1, str2):
        if str1==str2:
            return 0
        return compare_str_helper(str1, str2, 0)
        
    def compare_hands(hand1, hand2):
        # return if hand1 is greater than hand2
        if hand1 == hand2:
            return 0
        h_rank1, h_rank2 = get_hand_rank(hand1),get_hand_rank(hand2)
        if h_rank1==h_rank2:
            return int(compare_str(hand1, hand2))
        else:
            return int(h_rank1 - h_rank2)   
    
    def solve_1():
        nonlocal with_joker
        with_joker = False
        sorted_list = sorted(hand_map.keys(), key=cmp_to_key(compare_hands), reverse=True)
        return sum(map(lambda x: (x[0]+1) * hand_map[x[1]], enumerate(sorted_list)))
    
    def solve_2():
        nonlocal with_joker
        with_joker = True
        sorted_list = sorted(hand_map.keys(), key=cmp_to_key(compare_hands), reverse=True)
        return sum(map(lambda x: (x[0]+1) * hand_map[x[1]], enumerate(sorted_list)))

    return [solve_1, solve_2]
_ = solve_day()


Answer for Ques 7
Part 1:	250898830
Part 2:	252127335


In [13]:
# Day 7

@assert_day(day=7)
@print_day(day=7)
def solve_day():
    data  = read_input(DAY)

    hand_map = {x.split(' ')[0]:int(x.split(' ')[1]) for x in data}
    cards = ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3',  '2']
    cards_jokers = ['A', 'K', 'Q', 'T', '9', '8', '7', '6', '5', '4', '3',  '2', 'J']
    cards_rank = dict(zip(cards, range(len(cards))))
    cards_rank_joker = dict(zip(cards_jokers, range(len(cards_jokers))))
    with_joker = False

    def resolve_joker(hand):
        counts = Counter(hand)
        sorted_counts = sorted(counts, key=counts.get, reverse=True)
        resovled_char = 'J'
        for i in sorted_counts:
            if i !='J':
                resovled_char = i
                break
        return hand.replace('J', resovled_char)

    def get_hand_rank(hand):
        nonlocal with_joker
        if with_joker:
            hand = resolve_joker(hand)

        sorted_lst = sorted(Counter(hand).values())
        if sorted_lst == [ 5 ]:
            return 1
        elif sorted_lst == [1,4]:
            return 2
        elif sorted_lst == [2,3]:
            return 3
        elif sorted_lst == [1,1,3]:
            return 4
        elif sorted_lst == [1,2,2]:
            return 5
        elif sorted_lst == [1,1,1,2]:
            return 6
        else:
            return 7

    def compare_str_helper(str1, str2, idx):
        nonlocal with_joker
        ranks = cards_rank_joker if with_joker else cards_rank
        if idx >= min(len(str1), len(str2)):
            return 0
        r1, r2 = ranks[str1[idx]], ranks[str2[idx]]
        return r1 - r2 if r1!=r2 else compare_str_helper(str1, str2, idx+1)

    def compare_str( str1, str2):
        if str1==str2:
            return 0
        return compare_str_helper(str1, str2, 0)
        
    def compare_hands(hand1, hand2):
        # return if hand1 is greater than hand2
        if hand1 == hand2:
            return 0
        h_rank1, h_rank2 = get_hand_rank(hand1),get_hand_rank(hand2)
        if h_rank1==h_rank2:
            return int(compare_str(hand1, hand2))
        else:
            return int(h_rank1 - h_rank2)   
    
    def solve_1():
        nonlocal with_joker
        with_joker = False
        sorted_list = sorted(hand_map.keys(), key=cmp_to_key(compare_hands), reverse=True)
        return sum(map(lambda x: (x[0]+1) * hand_map[x[1]], enumerate(sorted_list)))
    
    def solve_2():
        nonlocal with_joker
        with_joker = True
        sorted_list = sorted(hand_map.keys(), key=cmp_to_key(compare_hands), reverse=True)
        return sum(map(lambda x: (x[0]+1) * hand_map[x[1]], enumerate(sorted_list)))

    return [solve_1, solve_2]
_ = solve_day()


Answer for Ques 7
Part 1:	250898830
Part 2:	252127335


In [14]:
# Day 8

@assert_day(day=8)
@print_day(day=8)
def solve_day():
    data = read_input(DAY)
    steps = data[0]
    nodes = list(map(lambda x: re.findall("[A-Z]{3}", x), data[2:]))
    node_map = {x[0]:x[1:] for x in nodes}

    def find_len(start:str, end:str)-> int:
        i, l, n =0, 0, len(steps)
        pairs = set()
        while(start!=end):
            pair = (i,start)
            if pair in pairs:
                return -1
            pairs.add(pair)
            # print(pair)
            start = node_map[start][0] if steps[i] == 'L' else node_map[start][1]
            l+=1
            i = (i+1)%n
        return l

    def solve_1():
        return find_len('AAA', 'ZZZ')

    def solve_2():
        starts = [ x for x in node_map.keys() if x[2]=='A']
        ends = [ x for x in node_map.keys() if x[2]=='Z']
        lengths = []
        for s in starts:
            elem = list(filter(lambda x: x !=-1, map(lambda x: find_len(s,x), ends)))
            lengths.append(elem)
        return min(map(lambda x: math.lcm(*x), itertools.product(*lengths)))
    
    return [solve_1, solve_2]
_ = solve_day()

Answer for Ques 8
Part 1:	19631
Part 2:	21003205388413


In [15]:
# Day 9

@assert_day(day=9)
@print_day(day=9)
def solve_day():
    data = read_input(DAY)

    diffs = lambda data: list(map(lambda x: x[1]-x[0], zip(data[:-1], data[1:])))
    def solver(elem, end=1):
        elem = list(map(int, elem.split()))
        return predict_end(elem) if end else predict_start(elem)

    def predict_end(data):
        elems = diffs(data)
        if all(v == 0 for v in elems):
            return data[-1]
        return predict_end(elems)+data[-1]

    def predict_start(data):
        elems = diffs(data)
        if all(v == 0 for v in elems):
            return data[0]
        return data[0] - predict_start(elems)

    def solve_1():
        return sum(map(lambda x: solver(x,1), data))

    def solve_2():
        return sum(map(lambda x: solver(x,0), data))
        
    return [solve_1, solve_2]
_ = solve_day()

Answer for Ques 9
Part 1:	1974232246
Part 2:	928


In [16]:
# Day 10

@assert_day(day=10)
@print_day(day=10)

def solve_day():
    data = read_input(DAY)
    data = list(map(list, data))
    types = ['-','|', 'F', '7', 'L','J']
    n,m = len(data), len(data[0])
    sys.setrecursionlimit(n*m+1)
    moves = {
        '|': [ (1,0), (-1,0)],
        'F': [ (1,0), (0,1) ],
        'J': [ (-1,0),(0,-1)],
        'L': [ (-1,0),(0,1) ],
        '7': [ (1,0), (0,-1)],
        '-': [ (0,-1),(0,1) ]
    }

    visited = set()
    #  to decide parent -> node -> next node
    new_moves = {k:{v[0]:v[1], v[1]:v[0]} for k,v in moves.items()}

    # Find S
    start = list(
                filter(
                    lambda x: x!=(-1,-1), 
                    map(
                        lambda x: 
                            (x[0], x[1].index('S')) if 'S' in x[1] else (-1,-1), 
                        enumerate(data)
                        )
                    )
                )[0]

    move_next = lambda start, move: (start[0]+move[0], start[1]+move[1])
    check     = lambda node: node[0]>=0 and node[0]<n and node[1]>=0 and node[1]<m
    get_data  = lambda x: data[x[0]][x[1]]


    def dfs(node, parent, type):
        nonlocal visited
        node_type = get_data(node)
        # if visited node, then cross connection hence not the main loop
        if node in visited or node_type == '.':
            return -1
        
        # visit node
        visited.add(node)
        last_move = (-node[0]+parent[0], -node[1]+parent[1])
        next_moves = new_moves[node_type]

        # Check is the path from parent to node is a valid path
        if last_move in next_moves:
            # if we reached start then return from dfs
            if node==start:
                return 0
            
            next_node = move_next(node, next_moves[last_move])
            
            if not check(next_node):
                return -1
            dis = dfs(next_node, node,  type)
            return -1 if dis == -1 else dis+1
        else:
            return -1


    def run_from_start(start,  type):
        nonlocal visited, data
        visited = set()
        data[start[0]][start[1]] = type
        move = moves[type][0]
        next_node = move_next(start, move)
        ans = -1 if not check(next_node) else dfs(next_node, start,  type)
        data[start[0]][start[1]] = 'S'
        return ans

    def find_inner_points(start, type):
        if run_from_start(start, type)==-1:
            return []
        ans = []
        for i in range(n):
            count = 0
            cur_dir = 0
            for j in range(m):
                if (i,j) in visited:
                    if data[i][j]=='|':
                        count+=1
                    elif data[i][j] == 'F'  :
                        cur_dir = -1
                    elif data[i][j] == 'L':
                        cur_dir = 1
                    elif data[i][j] == '7':
                        count += 2 if cur_dir == -1 else 1
                        cur_dir = 0
                    elif data[i][j] == 'J' or data[i][j] == 'S':
                        count += 1 if cur_dir == -1 else 2
                        cur_dir = 0
                else:
                    if count%2:
                        ans.append((i,j))
        return ans
        

    def solve_1():
        dis = list(map(lambda x: run_from_start(start, x), types))
        return math.ceil(max(dis)/2)
    
    def solve_2():
        ans = list(map(lambda x: find_inner_points(start, x), types))
        return max(map(len, ans))
    
    return [solve_1, solve_2]
_ = solve_day()

Answer for Ques 10
Part 1:	6927
Part 2:	467


In [17]:
# Day 11

@assert_day(day=11)
@print_day(day=11)
def solve_day():
    data  = read_input(DAY)
    n,m = len(data), len(data[0])
    all_dot_row = "".join(['.']*m)

    rows = [i for i, x in enumerate(data) if x == all_dot_row]
    cols = [j for j in range(m) if all(data[i][j]=='.' for i in range(n))]
    galaxies = [ (i,j) for i, row in enumerate(data) for j, elem in enumerate(row) if data[i][j]=='#']
    pairs = [(i,j) for i in range(len(galaxies)) for j in range(i+1, len(galaxies))]
    
    def min_distance(galaxy, factor=2)->int:
        g1, g2 = galaxies[galaxy[0]], galaxies[galaxy[1]]
        dis = abs(g1[0]-g2[0]) + abs(g1[1]-g2[1])
        g1_a, g2_a = min(g1[0],g2[0] ), max(g1[0],g2[0] )
        g1_b, g2_b = min(g1[1],g2[1] ), max(g1[1],g2[1] )
        dis += len([_ for x in rows if x > g1_a and x < g2_a])*(factor-1) \
                + len([_ for x in cols if x > g1_b and x < g2_b])*(factor-1)
        return dis

    def solve_1():
        return sum(map(min_distance, pairs))

    def solve_2():
        return sum(map(lambda x: min_distance(x, 1000000), pairs))
        
    return [solve_1, solve_2]
_ = solve_day()

Answer for Ques 11
Part 1:	9521776
Part 2:	553224415344


In [18]:
# Day 12

@assert_day(day=12)
@print_day(day=12)
def solve_day():
    data = read_input(DAY)
    def parse(elem):
        [pattern, nums] = elem.split(" ")
        nums = list(map(int, nums.split(',')))
        return (list(pattern), nums)
    data = list(map(parse, data))

    def elem_dp_helper(pattern:List[str], nums:List[int], i:int, j:int, dp_dict:dict)->int:
        if (i,j) in dp_dict:
            return dp_dict[(i,j)]
        ans = 0
        if i <0 :
            ans =  1 if j < 0 else 0
        elif pattern[i] == '.':
            ans =  elem_dp_helper(pattern, nums, i-1, j, dp_dict) if i else (1 if j<0 else 0)
        elif pattern[i] == '#' or pattern[i] == '?':
            if j<0:
                ans = 0
            elif sum(map( lambda x: x!='.', pattern[:i+1][-nums[j]:] ) ) == nums[j] and (i<nums[j] or pattern[i-nums[j]]!='#'):
                ans = elem_dp_helper(pattern, nums, i-nums[j]-1, j-1,dp_dict)
            ans += elem_dp_helper(pattern, nums, i-1, j, dp_dict)  if pattern[i] == '?' else 0
        dp_dict[(i,j)] = ans
        return ans

    def elem_dp(input:tuple, multiple:int=1) -> int:
        [pattern, nums] = input
        pattern, nums = ((pattern + ['?'])*multiple)[:-1], nums*multiple
        dp_dict = dict()
        return elem_dp_helper(pattern, nums, len(pattern)-1, len(nums)-1, dp_dict)

    def solve_1():
        return sum(map(elem_dp, data))
    def solve_2():
        return sum(map(lambda x: elem_dp(x,5), data))
        
    return [solve_1, solve_2]
_ = solve_day()

Answer for Ques 12
Part 1:	7047
Part 2:	17391848518844


In [19]:
# Day 13

@assert_day(day=13)
@print_day(day=13)
def solve_day():
    data         = read_input(DAY)
    data         = (" ".join(data)).split("  ")
    mats         = list(map(lambda x: list(map(tuple, x.split(' '))), data ))
    encode       = lambda elem: sum(map(lambda x: 2**(x[0])*(x[1]=='#'), enumerate(elem)))
    transpose    = lambda mat: list(zip(*mat))
    arrayDiff    = lambda x,y: [i^j for (i,j) in zip(x,y) if i!=j ]
    isPowerOfTwo = lambda x: (x and (not(x & (x - 1))) )

    def check_mirror(l1, l2):
        n1, n2 = len(l1), len(l2)
        diff = arrayDiff(l1[-n2:], l2[::-1]) if n1>=n2 else arrayDiff(l1[::-1], l2[:n1])
        return diff

    def find_above(mat):
        encoding = list(map(encode, mat))
        for i in range(len(encoding)-1):
            if encoding[i]==encoding[i+1] and len(check_mirror(encoding[:i+1], encoding[i+1:]))==0:
                return i+1
        return 0
        
    def find_left(_mat):
        return find_above(transpose(_mat))

    def find_above_with_smuge(mat):
        encoding = list(map(encode, mat))
        
        for i in range(len(encoding)-1):
            diff = check_mirror(encoding[:i+1], encoding[i+1:])
            if len(diff) == 1 and isPowerOfTwo(diff[0]):
                    return i+1
        return 0
                
    def find_left_with_smuge(_mat):
        return find_above_with_smuge(transpose(_mat))              

    def solve_1():
        return sum(map(lambda x: find_above(x)*100 + find_left(x), mats))

    def solve_2():
        return sum(map(lambda x: find_above_with_smuge(x)*100 + find_left_with_smuge(x), mats))
    return [solve_1, solve_2]
_ = solve_day()

Answer for Ques 13
Part 1:	33728
Part 2:	28235
